In [1]:
#   >>> import nltk
#   >>> nltk.download('omw-1.4')

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

In [3]:
import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2


In [4]:
data = pd.read_csv('../../raw_data/raw_en.csv')

/tmp/ipykernel_23299/359733226.py:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../../raw_data/raw_en.csv')


In [20]:
data.head()

,name,date,recommendation,review,playing_hours,source,stars,language
0,Expansion - Hearts of Iron IV: Man the Guns,2019-02-10,Recommended,&gt Played as German Reich&gt Declare war on B...,578.0,steam,NaN,en
1,Expansion - Hearts of Iron IV: Man the Guns,2019-02-07,Recommended,Very good game although a bit overpriced in my...,892.0,steam,NaN,en
2,Dead by Daylight,2018-06-14,Recommended,Out of all the reviews I wrote This one is pro...,676.0,steam,NaN,en
3,Dead by Daylight,2017-06-20,Recommended,Disclaimer I survivor main. I play games for f...,612.0,steam,NaN,en
4,Dead by Daylight,2016-12-12,Recommended,ENGLISH After playing for more than two years ...,2694.0,steam,NaN,en


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457074 entries, 0 to 457073
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   name            457074 non-null  object 
 1   date            351179 non-null  object 
 2   recommendation  350678 non-null  object 
 3   review          457074 non-null  object 
 4   playing_hours   350678 non-null  float64
 5   source          457074 non-null  object 
 6   stars           106396 non-null  float64
 7   language        457074 non-null  object 
dtypes: float64(2), object(6)
memory usage: 27.9+ MB


In [21]:
data.recommendation.head(1000).value_counts()

Recommended        673
Not Recommended    327
Name: recommendation, dtype: int64

In [22]:
data.recommendation.isnull().sum()

0

In [9]:
data.stars.isnull().sum()

350678

In [9]:
#data['recommendation'] = data.apply(lambda x: 'Recommended' if data['stars']>3,columns=['recommendation'])

In [10]:
data['stars'] != np.nan 

0         True
1         True
2         True
3         True
4         True
          ... 
457069    True
457070    True
457071    True
457072    True
457073    True
Name: stars, Length: 457074, dtype: bool

In [18]:
def convert_stars(row):
    if pd.isna(row['stars']) == False:
        if row['stars'] > 3:
            return 'Recommended'
        return 'Not Recommended'
    return row['recommendation']


In [23]:
data['recommendation'] = data.apply(convert_stars, axis = 1)

In [24]:
data.recommendation.isnull().sum()

0

In [25]:
X = data[['review']].copy()
y = data['recommendation']

In [26]:
def remove_punctuations(text):
    punctuations = string.punctuation
    for punctuation in punctuations:
        text = text.replace(punctuation, '')
    return text

def lowercase(text): 
    text = text.lower() #str was removed
    return text

def remove_num(text):
    text = ''.join(word for word in text if not word.isdigit())
    return text

# def lemmatize(text):
#     nlp = spacy.load("en_core_web_sm")
#     text = ' '.join(token.lemma_ for token in nlp(text))
#     return text

def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(text)
    lemmatized = [lemmatizer.lemmatize(word) for word in word_tokens]
    return ' '.join(lemmatized)

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    word_tokens = [word for word in word_tokens if not word in stop_words]
    return ' '.join(word_tokens)

def text_prepro(text): # this does the whole job. 
    text = lowercase(text)
    text = remove_punctuations(text)
    text = remove_num(text)
    text = lemmatize(text)
    text = remove_stopwords(text)
    return text

def series_prepro(series): # this function does not work, what is the objective of it?
    return pd.DataFrame(series.apply(text_prepro))


In [27]:
X.review[3]

"Disclaimer I survivor main. I play games for fun not for competition so the DBD community doesn't really get to me. If I get a bad killer that face camps oh well. I die and move on to the next game. If I get a good killer who finds me immediately hooks me (bc I can't juke whoops) and then patrol camps (which I'm 100% OK with) then no biggie. Next game please I think a lot of the community salt from killers comes from dealing with rank one survivors that don't actually play the game properly and just run around and flash the killer in the face. And while that is a legitimate strategy it takes away from the fun of the game in my opinion.As a killer (a horrible one) I've also been called names by survivors who tell me to 'get gud.' Older players ruin it for newer players or players who don't put in 1000s of hours. Just relax and have fun with the game! So what if you don't pip up there is always the next game )That being said this game is crazy fun. Don't let the saltiness of the communi

In [35]:
X['review_clean'] = X['review'].apply(text_prepro)

In [36]:
X['review_clean'][3]

'disclaimer survivor main play game fun competition dbd community doesnt really get get bad killer face camp oh well die move next game get good killer find immediately hook bc cant juke whoop patrol camp im ok biggie next game please think lot community salt killer come dealing rank one survivor dont actually play game properly run around flash killer face legitimate strategy take away fun game opinionas killer horrible one ive also called name survivor tell get gud older player ruin newer player player dont put hour relax fun game dont pip always next game said game crazy fun dont let saltiness community keep buying game grab fun ignore anyone might give hard time dont fixate much ranking process unless youre absurdly highly competitive reason also fine try let push spout anger even recognize jerk play game like game fun compete'

In [30]:
X['review_clean'].head()

0    gt played german reichgt declare war belgiumgt...
1    good game although bit overpriced opinion id p...
2    review wrote one probably serious one wrote st...
3    disclaimer survivor main play game fun competi...
4    english playing two year given task reviewing ...
Name: review_clean, dtype: object

## Latent Dirichlet Allocation model

In [57]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def comp_iter(n):
 
    vectorizer = TfidfVectorizer().fit(X['review_clean'].head(100000))

    data_vectorized = vectorizer.transform(X['review_clean'].head(100000))

    lda_model = LatentDirichletAllocation(n_components=n).fit(data_vectorized)
    
    return lda_model

In [61]:
vectorizer = TfidfVectorizer().fit(X['review_clean'].head(1000))

data_vectorized = vectorizer.transform(X['review_clean'].head(1000))

lda_model = LatentDirichletAllocation(n_components=n).fit(data_vectorized)

def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])

print_topics(lda_model, vectorizer)

Topic 0:
[('game', 19.323242751676784), ('fun', 15.052526717936516), ('good', 14.376000491196406), ('great', 12.7236597382293), ('friend', 12.385800351314412), ('pretty', 11.722256767252109), ('job', 9.18633729529896), ('play', 9.106062289077363), ('far', 6.699383740707762), ('really', 6.034004771868148)]
Topic 1:
[('addicted', 1.6599403851480143), ('meme', 1.0457214262902121), ('badge', 0.9539582874064797), ('billy', 0.9379316991983984), ('atmospheric', 0.9213042016390174), ('alright', 0.9095288834922292), ('best', 0.8039942483358649), ('rip', 0.7901702553050035), ('beast', 0.7588784018477108), ('wwwredditcomrdeadbydaylightcommentsnqpodiscordserverfordbdtalking', 0.7093230413573398)]
Topic 2:
[('addictive', 2.1532162857437265), ('terrifying', 2.131637652857732), ('join', 2.126681814720185), ('button', 1.385623432373373), ('crash', 1.2917264673647144), ('level', 1.276656065327776), ('win', 1.237659678294244), ('hatch', 1.2094509876098294), ('havent', 1.0890916952099623), ('couldnt', 1.

In [68]:
vectorizer = TfidfVectorizer().fit(X['review_clean'].head(100000))

data_vectorized = vectorizer.transform(X['review_clean'].head(100000))

lda_model = LatentDirichletAllocation(n_components=50).fit(data_vectorized)

def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])

print_topics(lda_model, vectorizer)

Topic 0:
[('great', 1662.194064927577), ('game', 904.173153844072), ('suck', 240.58139840312472), ('friend', 228.39307640433978), ('play', 226.32116198460483), ('fun', 220.69455026749935), ('wow', 107.90980472157516), ('fantastic', 100.37451670323748), ('controller', 88.51981515364567), ('good', 84.005933013724)]
Topic 1:
[('perfect', 121.58283466475079), ('goal', 64.45589809587143), ('score', 57.78307080368131), ('aight', 54.611480935035424), ('fly', 53.419653238885424), ('exciting', 51.046984477779105), ('would', 29.99043807112224), ('meme', 28.7348048167768), ('game', 24.40490452576007), ('scored', 22.741964136125095)]
Topic 2:
[('easy', 357.16617569847665), ('hard', 345.99725438967334), ('master', 338.2380906560277), ('learn', 228.18896619583393), ('fun', 166.40212709851863), ('game', 149.6172894296822), ('simple', 129.89416325414965), ('play', 117.39888781682214), ('pick', 108.16621083755568), ('difficult', 90.07086881105708)]
Topic 3:
[('life', 41.761235348837566), ('game', 32.17

[('lt', 246.10657161488842), ('pure', 35.71479172432259), ('crash', 34.10797170272568), ('rock', 27.773129731004264), ('game', 22.524204626459873), ('crashing', 20.191003023376872), ('mm', 15.006457540493226), ('love', 12.158560507467158), ('approved', 10.228440493542378), ('badass', 8.369757807299582)]
Topic 27:
[('awsome', 179.0217180269995), ('plz', 39.06509812475691), ('addiction', 34.20580408622534), ('game', 20.086266279142414), ('optimization', 15.783858550667501), ('furious', 8.373589024541907), ('carswhat', 7.394875254318331), ('steroid', 7.359956972689051), ('lie', 6.927448548255711), ('friendsbut', 6.731709270974)]
Topic 28:
[('game', 922.6592224912971), ('best', 917.4737966912139), ('ever', 815.9002543943084), ('played', 609.1640418875453), ('one', 482.58191874770586), ('soccer', 438.1549735364326), ('ive', 435.49408381788254), ('car', 361.6957595268608), ('rocket', 359.978692859243), ('fun', 333.4408842599585)]
Topic 29:
[('year', 82.61651733508108), ('favourite', 71.01647

In [65]:
for n in range(10,100,10):
    lda_model = comp_iter(n)
    print(f'\n\n number of components = {n}')
    print_topics(lda_model, vectorizer)
    



 number of components = 10
Topic 0:
[('game', 3818.833736781737), ('fun', 2827.2260245145317), ('great', 2245.67426902934), ('play', 1822.1556218307483), ('friend', 1524.6043424801433), ('car', 1333.6088210706162), ('good', 1128.4652342053946), ('really', 988.0894642756656), ('rocket', 885.9750246942864), ('soccer', 832.0500515602126)]
Topic 1:
[('game', 3038.2930008665166), ('get', 1070.6023800301978), ('play', 1040.1003402170224), ('killer', 975.9620098053634), ('like', 958.7193973466027), ('fun', 950.2609270158692), ('time', 889.6213630531372), ('wa', 819.101806576157), ('monster', 805.2987108818216), ('good', 792.2826992743089)]
Topic 2:
[('trash', 78.09998989908607), ('sick', 74.78261157577005), ('aight', 55.29650449965983), ('gr', 46.90410741668199), ('neat', 21.72913906161052), ('game', 20.655442837017553), ('meme', 16.35223835278759), ('gamewhat', 14.48144438348789), ('boi', 13.684093741326821), ('bruh', 13.478320534688596)]
Topic 3:
[('best', 928.5343261576692), ('game', 592

[('game', 1186.9926394339507), ('server', 505.60263905798405), ('issue', 490.02478808047334), ('fix', 478.02129712288297), ('play', 462.8338680210085), ('good', 387.16840274403864), ('bug', 366.43365062505904), ('fun', 359.99985459387176), ('cant', 325.6129158225195), ('great', 325.1304118188592)]
Topic 18:
[('fun', 2228.6181769461605), ('game', 1938.1981342369147), ('play', 1314.3485496122198), ('friend', 1282.3584014973387), ('great', 784.3394578710643), ('really', 765.6758000343796), ('good', 703.8832764649927), ('amazing', 517.569597207207), ('recommend', 504.56111712598954), ('addicting', 416.58309804384646)]
Topic 19:
[('car', 807.2817911786601), ('soccer', 566.9612354285612), ('football', 421.951464103874), ('rocket', 348.8464886127661), ('like', 278.41004851267735), ('game', 259.30817141047294), ('lt', 227.33876317803777), ('ball', 225.15103326743022), ('league', 215.5811669489085), ('love', 169.13522832409245)]


 number of components = 30
Topic 0:
[('ign', 72.51999775219038),

[('stop', 107.90887777311), ('cant', 63.86992861839384), ('playing', 47.135408812454905), ('gg', 34.91708967141624), ('rip', 34.27594230501969), ('reading', 26.257261969940455), ('openiv', 25.389911923165524), ('game', 23.31430865556663), ('behalf', 18.039083062512773), ('iv', 16.070740655203284)]
Topic 25:
[('civ', 80.67729229179847), ('dead', 56.531389620545546), ('daylight', 33.128978552912955), ('battle', 32.113381690078434), ('acrobatic', 27.83319824650273), ('supersonic', 24.69603840457537), ('game', 23.773615406105083), ('civilization', 21.573080667835978), ('plz', 21.438656643586754), ('powered', 19.62172534146185)]
Topic 26:
[('alright', 188.34344240506002), ('gud', 125.92781397685202), ('monster', 98.83204251429592), ('guess', 96.20199509811287), ('hunter', 88.96856618198848), ('perfect', 86.10942855291455), ('yeah', 51.10142085430692), ('connection', 46.09974310232823), ('game', 38.9347025451241), ('fixed', 34.38383961178934)]
Topic 27:
[('game', 1841.043268269995), ('monste

KeyboardInterrupt: 

## Negative Comments - Frequent Words

In [48]:
neg_review = y == 'Not Recommended'
X_neg = X['review_clean'].head(100000).loc[neg_review]

In [51]:
X[neg_review].head(100000) 

,review,review_clean
12,Amount of bugs and glitches from error during ...,amount bug glitch error launching killer consi...
19,i would not recommend this.. this game used to...,would recommend game used fun fed pandered muc...
26,I couldn't even start a gamewith My friends. O...,couldnt even start gamewith friend brightside ...
27,The game is completely unbalanced it doesn't m...,game completely unbalanced doesnt make sense t...
33,this game is dog s hit,game dog hit
...,...,...
293611,For the price its not their yet. Not even clos...,price yet even close organic gameplay check bu...
293618,I have played this game for a while now on the...,played game toronto developement server feel l...
293623,Not recommended for now It is a great game gre...,recommended great game great concept great mec...
293624,Game is crap ever since last update no sense p...,game crap ever since last update sense playing...


In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer


# Tuned TFidfvectorizer
vec = TfidfVectorizer(ngram_range = (2,2), min_df=0.01, max_df = 0.05).fit(X[neg_review]['review_clean'].head(100000))

vectors = vec.transform(X[neg_review]['review_clean'].head(100000)) # Transform text to vectors

sum_tfidf = vectors.sum(axis=0) # Sum of tfidf weighting by word

tfidf_list = [(word, sum_tfidf[0, idx]) for word, idx in     vec.vocabulary_.items()]  # Get the word and associated weight

sorted_tfidf_list =sorted(tfidf_list, key = lambda x: x[1], reverse=True)  # Sort

sorted_tfidf_list

[('single player', 3023.4995554808047),
 ('buy game', 2585.813833552124),
 ('play game', 2558.2219673997784),
 ('dont buy', 2487.055826529692),
 ('game ha', 2434.6503539605287),
 ('early access', 1923.7003358030295),
 ('game wa', 1855.6589647161238),
 ('good game', 1758.559457466914),
 ('received free', 1733.420734743786),
 ('product received', 1730.9017006170977),
 ('recommend game', 1637.0969073323483),
 ('shark card', 1593.285321062681),
 ('gta online', 1579.4688820800047),
 ('great game', 1545.4064919296354),
 ('playing game', 1322.518750540879),
 ('fun game', 1321.547356696594),
 ('make game', 1262.6858511475177),
 ('cant even', 1256.2757554477819),
 ('bought game', 1247.7797151306963),
 ('game still', 1231.594708764096),
 ('game fun', 1218.8078780351802),
 ('feel like', 1180.1215093715464),
 ('game ever', 1092.5987271799952),
 ('waste money', 1067.0590448789544),
 ('game dont', 1029.3276019564585),
 ('game play', 1007.2534031434707),
 ('want play', 991.1968885651208),
 ('like gam

## Positive Comments - Frequent Words

In [66]:
pos_review = y != 'Not Recommended'
X_pos = X['review_clean'].head(10000).loc[pos_review]

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer


# Tuned TFidfvectorizer
vec = TfidfVectorizer(ngram_range = (2,2), min_df=0.01, max_df = 0.05).fit(X[pos_review]['review_clean'].head(10000))

vectors = vec.transform(X[pos_review]['review_clean'].head(10000)) # Transform text to vectors

sum_tfidf = vectors.sum(axis=0) # Sum of tfidf weighting by word

tfidf_list = [(word, sum_tfidf[0, idx]) for word, idx in     vec.vocabulary_.items()]  # Get the word and associated weight

sorted_tfidf_list =sorted(tfidf_list, key = lambda x: x[1], reverse=True)  # Sort

sorted_tfidf_list

[('love game', 317.5982752688739),
 ('dead daylight', 277.0484518744964),
 ('game play', 271.1226194018729),
 ('game fun', 262.0695259875463),
 ('game ha', 259.27097109128385),
 ('fun play', 239.13387022821348),
 ('pretty good', 213.76450735078504),
 ('really fun', 210.55706200870895),
 ('horror game', 192.75697047665687),
 ('killer survivor', 185.1788349247908),
 ('nice game', 181.67308885702403),
 ('play game', 180.5333393326819),
 ('game good', 170.76104724673544),
 ('best game', 160.94175940924347),
 ('like game', 159.27338701304424),
 ('game really', 157.58582029601322),
 ('recommend game', 156.5590512166424),
 ('survivor killer', 152.84573158877464),
 ('game great', 148.59348940766054),
 ('good job', 141.74967002587758),
 ('lot fun', 130.7073880612113),
 ('playing game', 125.90712846806916),
 ('really good', 121.06730248216354),
 ('fun friend', 119.7694244801003),
 ('game would', 118.05267427403163),
 ('play killer', 114.40923878069391),
 ('game like', 111.82541403034244),
 ('gam